In [2]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [3]:
def MAPE_regression(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [62]:
#NuralNet (Regression)
class RegressionNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        
        return {"Predarray" :pred_x}

    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))
                
   
        

In [63]:
#NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        
        return {"Predarray" :pred_x}

    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [70]:

def CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    #신경망 생성하기
    model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "CNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    print(pred_y_test)
    pred_y_test_tensor=pred_y_test['Predarray']
    print(pred_y_test_tensor)
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test_tensor = pred_y_test_tensor.detach().numpy()
    
      # calculate AUC of model
    fpr, tpr, thresholds = roc_curve(y_test, pred_y_test_tensor)
    auc = roc_auc_score(y_test, pred_y_test_tensor)
      
    return  {"auc_score": auc, "fpr":fpr, "tpr":tpr}
    

In [71]:
def RNet(X_train, y_train, X_test, y_test , hiddenlayer, node, Ouputnode, epochs):
    
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 신경망 생성하기
    model = RegressionNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
            
    #학습되 모델 저장
    torch.save(model, "RNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    pred_y_test_tensor = pred_y_test['Predarray'] 
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
    MSE = mean_squared_error(y_test, pred_y_test)  # MSE 값은 출력하지 않아도 됩니다. RMSE를 구하기 위한 절차입니다.
    
    # 출력 대상입니다. 성능지표로써 3가지값이 출력되게 됩니다.
    RMSE = np.sqrt(MSE)
    MAPE = MAPE_regression(y_test, pred_y_test)
    R_squared = r2_score(y_test, pred_y_test)
    return {'pred_y' : pred_y_test, 'RMSE' : RMSE, 'MAPE' : MAPE, "R_squared" : R_squared}

In [97]:
# ANN Model load

def ANN_model_loader(pickleFile, tsdat) :
    """
    저장한 모델을 로드한 후, 로드한 모델과 데이터를 활용해 분석 결과 리턴

    Parameters
    ----------
    model : ?
        로드한 모델
    tsdat : array
        예측 데이터

    Returns
    -------
    모델 리턴과 동일

    """
    model = torch.load(pickleFile)  
    X = torch.Tensor(tsdat.values)
    pred = model.predict(X)
    pred = pred['Predarray']
    pred = pred.detach().numpy()
    
    
    return  pred 



In [73]:
df1 = pd.read_csv("ab_te.csv")
df2 = pd.read_csv("RMS_bearing.csv")

In [134]:
x = df1.iloc[:,[0, 1, 2, 3]]
y = df1.iloc[:,[4]]
x1 = df2.iloc[:,[0,1,2]]
y1 = df2.iloc[:,[3]]


X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

X1_train,X1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=33)

In [75]:
pred_y2_test= CNet(X_train, y_train, X_test,y_test, 2, 5, 1, 10)



#CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):

0epoch // loss =-0.0
{'Predarray': tensor([[-0.4317],
        [-0.5106],
        [-0.5007],
        [-0.5086],
        [-0.4814],
        [-0.4757],
        [-0.4785],
        [-0.5348],
        [-0.6448],
        [-0.5165],
        [-0.4891],
        [-0.6492],
        [-0.6250],
        [-0.4872],
        [-0.4969],
        [-0.5269],
        [-0.5597],
        [-0.4945],
        [-0.4983],
        [-0.5240],
        [-0.4544],
        [-0.4870],
        [-0.5340],
        [-0.4593],
        [-0.5463],
        [-0.5106],
        [-0.5900],
        [-0.5268],
        [-0.6246],
        [-0.5116],
        [-0.4773],
        [-0.5393],
        [-0.4965],
        [-0.4788],
        [-0.6392],
        [-0.4845],
        [-0.5137],
        [-0.6335],
        [-0.5255],
        [-0.5123],
        [-0.6584],
        [-0.5119],
        [-0.4583],
        [-0.4928],
        [-0.6693],
        [-0.5044],
        [-0.4800],
        [-0.5839],
        [-0.5247],
        [-0.4904],
        [-0.468

In [76]:
print(pred_y2_test)

{'auc_score': 0.6908168935815504, 'fpr': array([0.        , 0.        , 0.        , 0.00819672, 0.00819672,
       0.01639344, 0.01639344, 0.02459016, 0.02459016, 0.03278689,
       0.03278689, 0.04098361, 0.04098361, 0.04918033, 0.04918033,
       0.05737705, 0.05737705, 0.08196721, 0.08196721, 0.10655738,
       0.10655738, 0.1147541 , 0.1147541 , 0.13114754, 0.13114754,
       0.13934426, 0.13934426, 0.14754098, 0.14754098, 0.1557377 ,
       0.1557377 , 0.18032787, 0.18032787, 0.20491803, 0.20491803,
       0.22131148, 0.22131148, 0.2295082 , 0.2295082 , 0.23770492,
       0.23770492, 0.24590164, 0.24590164, 0.2704918 , 0.2704918 ,
       0.27868852, 0.27868852, 0.28688525, 0.28688525, 0.29508197,
       0.29508197, 0.31147541, 0.31147541, 0.32786885, 0.32786885,
       0.33606557, 0.33606557, 0.36065574, 0.36065574, 0.36885246,
       0.36885246, 0.37704918, 0.37704918, 0.3852459 , 0.3852459 ,
       0.41803279, 0.41803279, 0.45081967, 0.45081967, 0.46721311,
       0.46721311, 0.

In [69]:
pred_y1_test = RNet(X1_train, y1_train, X1_test, y1_test, 2, 5, 1, 10)


0epoch // loss =1.4847279089735821e-05


AttributeError: 'dict' object has no attribute 'detach'

In [ ]:
print(pred_y1_test)

In [ ]:
RNET = torch.load("RNet.pt")
RNET.eval()

In [52]:
CNET = torch.load("CNet.pt")
CNET.eval()

ClassificationNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=4, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [121]:
pred = ANN_model_loader("RNet.pt", X1_test) 
pred2 = ANN_model_loader("CNet.pt", X_test) 

In [123]:
print(pred2)

[[-0.431688  ]
 [-0.51064414]
 [-0.5007413 ]
 [-0.5085681 ]
 [-0.48137718]
 [-0.47569665]
 [-0.478481  ]
 [-0.53478116]
 [-0.64481914]
 [-0.5164831 ]
 [-0.48913002]
 [-0.64923143]
 [-0.624988  ]
 [-0.48717114]
 [-0.49688876]
 [-0.5269459 ]
 [-0.5596838 ]
 [-0.49450058]
 [-0.4982707 ]
 [-0.5240153 ]
 [-0.4544198 ]
 [-0.4870419 ]
 [-0.533968  ]
 [-0.45932218]
 [-0.5463269 ]
 [-0.5105579 ]
 [-0.5900216 ]
 [-0.5267583 ]
 [-0.62464166]
 [-0.5115942 ]
 [-0.47728044]
 [-0.5393333 ]
 [-0.4965279 ]
 [-0.47877356]
 [-0.6391877 ]
 [-0.4844889 ]
 [-0.5136964 ]
 [-0.6335081 ]
 [-0.5255253 ]
 [-0.5122563 ]
 [-0.65844977]
 [-0.51188   ]
 [-0.45833248]
 [-0.49279922]
 [-0.6692928 ]
 [-0.5044303 ]
 [-0.4799586 ]
 [-0.5838595 ]
 [-0.52473974]
 [-0.4903763 ]
 [-0.4688428 ]
 [-0.50462615]
 [-0.5484752 ]
 [-0.519225  ]
 [-0.4866736 ]
 [-0.4671256 ]
 [-0.49169773]
 [-0.5148054 ]
 [-0.58003795]
 [-0.5117135 ]
 [-0.5340092 ]
 [-0.48028278]
 [-0.43794397]
 [-0.50472754]
 [-0.48461336]
 [-0.48448232]
 [-0.58622

In [135]:
X_test = torch.Tensor(X_test.values)
pred_y_test = CNET.predict(X_test)
pred_y_test= pred_y_test['Predarray']
print(pred_y_test)

tensor([[0.1365],
        [0.1475],
        [0.1428],
        [0.1460],
        [0.1430],
        [0.1421],
        [0.1432],
        [0.1492],
        [0.1723],
        [0.1490],
        [0.1479],
        [0.1490],
        [0.1564],
        [0.1437],
        [0.1466],
        [0.1459],
        [0.1458],
        [0.1421],
        [0.1448],
        [0.1443],
        [0.1547],
        [0.1421],
        [0.1421],
        [0.1587],
        [0.1534],
        [0.1437],
        [0.1555],
        [0.1509],
        [0.1654],
        [0.1452],
        [0.1421],
        [0.1512],
        [0.1421],
        [0.1421],
        [0.1639],
        [0.1421],
        [0.1440],
        [0.1512],
        [0.1490],
        [0.1480],
        [0.1583],
        [0.1493],
        [0.1421],
        [0.1430],
        [0.1467],
        [0.1436],
        [0.1421],
        [0.1563],
        [0.1430],
        [0.1421],
        [0.1421],
        [0.1484],
        [0.1472],
        [0.1421],
        [0.1421],
        [0

In [126]:
x = pred_y_test.items()

print(x)

AttributeError: 'Tensor' object has no attribute 'items'

In [137]:
X_test = X_test.numpy()

In [138]:
print(X_test)

[[ 3.80602336e+00  7.63494074e-01 -9.77539539e-01  1.22985256e+00]
 [ 2.88958907e+00 -9.56516191e-02  7.65324593e-01 -9.44870412e-02]
 [-2.81016648e-01  1.17189419e+00  6.46489739e-01  1.38244259e+00]
 [-1.38564634e+00  1.03286839e+00  9.64946330e-01  1.18931937e+00]
 [-3.84974442e-02 -1.02425289e+00 -2.86061287e-01  1.26748371e+00]
 [ 9.19930637e-01  8.03538024e-01  6.98452473e-01  1.89077008e+00]
 [ 1.52581644e+00  1.26883960e+00  1.14634144e+00  1.60621655e+00]
 [ 8.93510342e-01 -7.05286086e-01  1.13732183e+00  1.85925201e-01]
 [-7.45542467e-01  1.20459169e-01  1.77639592e+00 -1.88747442e+00]
 [ 1.36878800e+00  1.86112916e+00  1.96436131e+00  9.19935822e-01]
 [ 2.69998002e+00  5.97405493e-01  1.54226673e+00  6.48979664e-01]
 [-8.96511495e-01 -6.84065670e-02 -1.07213056e+00 -1.85259724e+00]
 [-1.97832420e-01  1.87706971e+00  1.81530488e+00 -4.25111383e-01]
 [ 1.40733492e+00  8.21575403e-01  2.77685165e-01  3.94608140e-01]
 [ 1.38334978e+00 -1.24526672e-01  9.10424232e-01  6.20447576e